In [9]:
# import packages
import os, sys, glob
import numpy as np
import pandas as pd
import multiprocessing as mp

from functools import partial

sys.path.append('../')

In [10]:
# CHANGE FILE AND PARSE ON TAB
# load data such that each row is a sample with one column for name and one for sequence
def load_data(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
        seqs = []
        for i in range(0, len(lines)):
            seqs.append(lines[i].strip())
    return pd.DataFrame({'sequence': seqs})

seqs = load_data('../data/clustalo-I20240820-222424-0237-19311203-p1m.aln-fasta.oneline.txt.Residue_Columns.txt')
seqs.head()

,sequence
0,LQWWTCAYPR
1,VQWWTCAFPR
2,VQWWTCAFPR
3,VQWWTCAFPR
4,IQWWTCAFPR


In [14]:
# check sequence lengths
np.unique(seqs['sequence'].apply(len))

array([10])

In [12]:
# get list of unique amino acids
amino_acids = list(set(''.join(seqs['sequence'])))
amino_acids.sort()
len(amino_acids)

21

In [16]:
# SCORES / FEATURES are subject to change
# create a dictionary mapping amino acids to specific features
# first, define features we want to include

# 1. hydrophobicity (ordinal)
hydrophobic = ['A', 'C', 'I', 'L', 'M', 'F', 'V', 'W'] # 0
neutral = ['G', 'H', 'P', 'S', 'T', 'Y'] # 1
hydrophilic = ['R', 'N', 'D', 'Q', 'E', 'K'] # 2

hydrophobicity = {2: hydrophobic, 
                  1: neutral, 
                  0: hydrophilic}

# 2. volume (ordinal)
very_small = ['A', 'G', 'S'] #0
small = ['N', 'D', 'C', 'P', 'T'] #1
medium = ['Q', 'E', 'H', 'V'] #2
large = ['R', 'I', 'L', 'K', 'M'] #3
very_large = ['F', 'W', 'Y'] #4

volume = {0: very_small, 
          1: small, 
          2: medium, 
          3: large, 
          4: very_large}

# 3. chemical (arbitrary)
aliphatic = ['A', 'G', 'I', 'L', 'P', 'V']
aromatic = ['F', 'W', 'Y']
sulfur = ['C', 'M']
hydroxyl = ['S', 'T']
basic = ['R', 'H', 'K']
acidic = ['D', 'E']
amide = ['N', 'Q']

chemical = {'aliphatic': aliphatic, 
            'aromatic': aromatic, 
            'sulfur': sulfur, 
            'hydroxyl': hydroxyl, 
            'basic': basic, 
            'acidic': acidic, 
            'amide': amide}

# 4. charge (ordinal)
positive = ['R', 'H', 'K'] #1
negative = ['D', 'E'] #-1
uncharged = ['A', 'N', 'C', 'Q', 'G', 'I', 'L', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V'] #0

charge = {1: positive, 
          int(-1): negative, 
          0: uncharged}

# 5. hydrogen donor/acceptor (arbitrary)
# MAYBE CHANGE TO BINARY (BOND/NOT)
bond = ['R', 'K', 'W', 'D', 'E', 'N', 'Q', 'H', 'S', 'T', 'Y']
none = ['A', 'C', 'G', 'I', 'L', 'M', 'F', 'P', 'V']

hydrogen = {1: bond,
            0: none}

# 6. polarity (ordinal)
polar = ['R', 'N', 'D', 'Q', 'E', 'H', 'K', 'S', 'T', 'Y'] #1
nonpolar = ['A', 'C', 'G', 'I', 'L', 'M', 'F', 'P', 'W', 'V'] #0

polarity = {1: polar, 
            0: nonpolar}

# allows us to map amino acids to features, creating a (num_samples, seq_length, num_features) tensor
feature_list = [hydrophobicity, volume, chemical, charge, hydrogen, polarity]

In [19]:
amino_acids

['-',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y']

In [20]:
# create feature dictionary that contains the properties of each amino acid
feature_dict = {}

# loop through each amino acid and create a dictionary of features

# loop through amino acids
for aa in amino_acids:

    # '-' is a placeholder for padding - it has no features
    if aa == '-':
        features = [0, 0, 0, 0, 0, 0]
        feature_dict[aa] = features
    
    else:
        # make list of features for each amino acid
        features = []

        # loop through each feature
        for feature in feature_list:

            # loop through each key in the feature dictionary
            for key, list in feature.items():
                if aa in list:
                    features.append(key)
        feature_dict[aa] = features

# print dictionary for feature conversion
feature_dict

{'-': [0, 0, 0, 0, 0, 0],
 'A': [2, 0, 'aliphatic', 0, 0, 0],
 'C': [2, 1, 'sulfur', 0, 0, 0],
 'D': [0, 1, 'acidic', -1, 1, 1],
 'E': [0, 2, 'acidic', -1, 1, 1],
 'F': [2, 4, 'aromatic', 0, 0, 0],
 'G': [1, 0, 'aliphatic', 0, 0, 0],
 'H': [1, 2, 'basic', 1, 1, 1],
 'I': [2, 3, 'aliphatic', 0, 0, 0],
 'K': [0, 3, 'basic', 1, 1, 1],
 'L': [2, 3, 'aliphatic', 0, 0, 0],
 'M': [2, 3, 'sulfur', 0, 0, 0],
 'N': [0, 1, 'amide', 0, 1, 1],
 'P': [1, 1, 'aliphatic', 0, 0, 0],
 'Q': [0, 2, 'amide', 0, 1, 1],
 'R': [0, 3, 'basic', 1, 1, 1],
 'S': [1, 0, 'hydroxyl', 0, 1, 1],
 'T': [1, 1, 'hydroxyl', 0, 1, 1],
 'V': [2, 2, 'aliphatic', 0, 0, 0],
 'W': [2, 4, 'aromatic', 0, 1, 0],
 'Y': [1, 4, 'aromatic', 0, 1, 1]}

In [21]:
# convert sequences to feature tensors
def seq_to_tensor(seq, feature_dict):
    # initialize tensor
    tensor = []
    
    # loop through each amino acid in the sequence
    for aa in seq:
        # get features for amino acid
        features = feature_dict[aa]
        tensor.append(features)
        
    return np.array(tensor)


In [23]:
# use pool to apply conversion function to all sequences
with mp.Pool(mp.cpu_count()) as pool:
    seq_tensors = pool.map(partial(seq_to_tensor, feature_dict=feature_dict), seqs['sequence'])

In [26]:
# check that all sequences are the same length (THEY ARE NOT)
np.unique([len(seq) for seq in seq_tensors])

array([10])

In [27]:
# compute some norm between matrices (start with euclidian)``

In [ ]:
# use a clustering method to group sequences based on similarity